# Model

## cnn lstm

In [27]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

import torch
import torch.nn as nn
import pytorch_lightning as pl

class candle_cnn_multi_lstm_multi_class(pl.LightningModule):
    def __init__(
        self,
        input_dim,
        num_classes,
        kernels=[1, 3, 5],
        cnn_out_channels=32,
        fusion_out_channels=64,
        hidden_dim=128,
        num_lstm_layers=1,
        lr=1e-3,
        optimizer_name="adamw",
        optimizer_params=None,
        scheduler_name=None,
        scheduler_params=None,
        dropout=0.2,
    ):
        super().__init__()
        self.save_hyperparameters(ignore=["optimizer_params", "scheduler_params"])

        self.input_dim = input_dim
        self.num_classes = num_classes
        self.kernels = kernels
        self.num_branches = len(kernels)
        self.cnn_out_channels = cnn_out_channels
        self.fusion_out_channels = fusion_out_channels
        self.hidden_dim = hidden_dim
        self.num_lstm_layers = num_lstm_layers
        self.lr = lr
        self.optimizer_name = optimizer_name
        self.optimizer_params = optimizer_params or {}
        self.scheduler_name = scheduler_name
        self.scheduler_params = scheduler_params or {}
        self.dropout = dropout

        # --- Multi-branch convolutions ---
        branches = []
        for k in kernels:
            pad = (k - 1) // 2
            branches.append(
                nn.Sequential(
                    nn.Conv1d(input_dim, cnn_out_channels, kernel_size=k, padding=pad),
                    nn.BatchNorm1d(cnn_out_channels),
                    nn.ReLU(inplace=True),
                    nn.Dropout(dropout),
                )
            )
        self.branches = nn.ModuleList(branches)

        # --- Fusion conv2d ---
        self.fusion_conv2d = nn.Sequential(
            nn.Conv2d(self.num_branches, fusion_out_channels, kernel_size=(cnn_out_channels, 1)),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
        )

        # --- LSTM ---
        self.lstm = nn.LSTM(
            input_size=fusion_out_channels,
            hidden_size=hidden_dim,
            num_layers=num_lstm_layers,
            batch_first=True,
            dropout=dropout if num_lstm_layers > 1 else 0,
            bidirectional=False,
        )

        # --- Classifier ---
        self.fc = nn.Linear(hidden_dim, num_classes)

        # --- Loss ---
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        x_main = x["main"].transpose(1, 2)  # (B, input_dim, T)
        branch_outs = [branch(x_main) for branch in self.branches]  # list of (B, C, T)
        stacked = torch.stack(branch_outs, dim=1)  # (B, num_branches, C, T)
        fused = self.fusion_conv2d(stacked)       # (B, fusion_out_channels, 1, T)
        fused = fused.squeeze(2).transpose(1, 2)  # (B, T, fusion_out_channels)
        lstm_out, _ = self.lstm(fused)           # (B, T, hidden_dim)
        last_hidden = lstm_out[:, -1, :]         # (B, hidden_dim)
        logits = self.fc(last_hidden)            # (B, num_classes)
        return logits

    def training_step(self, batch, batch_idx):
        x, y, foo = batch
        logits = self(x)
        loss = self.criterion(logits, y)  # y: (B,) integer labels
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y,foo = batch
        logits = self(x)
        loss = self.criterion(logits, torch.argmax(y, dim=1))
        preds = torch.argmax(logits, dim=1)
        acc = (preds == torch.argmax(y, dim=1)).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        return {"val_loss": loss, "val_acc": acc}


    def configure_optimizers(self):
        optimizer_map = {
            "adamw": optim.AdamW,
            "adam": optim.Adam,
        }
        scheduler_map = {
            "reduce_on_plateau": lr_scheduler.ReduceLROnPlateau,
            "step_lr": lr_scheduler.StepLR,
            "onecycle": lr_scheduler.OneCycleLR,
        }

        optimizer_class = optimizer_map[self.optimizer_name]
        optimizer = optimizer_class(self.parameters(), lr=self.lr, **self.optimizer_params)

        if self.scheduler_name is None or self.scheduler_name.lower() == "none":
            return optimizer

        scheduler_class = scheduler_map[self.scheduler_name]

        if self.scheduler_name == "onecycle":
            scheduler = scheduler_class(
                optimizer,
                max_lr=self.lr,
                total_steps=self.trainer.estimated_stepping_batches,
                **self.scheduler_params,
            )
            return {
                "optimizer": optimizer,
                "lr_scheduler": {"scheduler": scheduler, "interval": "step"},
            }

        if self.scheduler_name == "reduce_on_plateau":
            scheduler = scheduler_class(optimizer, **self.scheduler_params)
            return {
                "optimizer": optimizer,
                "lr_scheduler": {"scheduler": scheduler, "monitor": "val_loss"},
            }

        scheduler = scheduler_class(optimizer, **self.scheduler_params)
        return [optimizer], [scheduler]


# Train

## cnn lstm

In [ ]:
import joblib
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from datetime import datetime
import pandas as pd
import io
import os
import numpy as np
from preprocess.classification_multilabel_dif_seq import preprocess_sequences_csv_multilabels
from utils.print_batch import print_batch
from utils.json_to_csv import json_to_csv_in_memory  # <-- new util
from utils.multilabel_threshold_tuning import tune_thresholds_nn
from add_ons.feature_pipeline5 import FeaturePipeline
from add_ons.drop_columns2 import drop_columns
from add_ons.candle_dif_rate_of_change_percentage2 import add_candle_rocp
from add_ons.candle_proportion import add_candle_proportions
from add_ons.candle_rate_of_change import add_candle_ratios
from add_ons.candle_proportion_simple import add_candle_shape_features
from add_ons.normalize_candle_seq import add_label_normalized_candles
from utils.make_step import make_step
from utils.padding_batch_reg import collate_batch
import numpy as np
import torch
from sklearn.metrics import f1_score, classification_report, multilabel_confusion_matrix

def evaluate_model(model, dataloader, threshold=0.5, multi_label=True, return_probs=False, return_preds=False):
    """
    Evaluate a PyTorch Lightning model on a dataloader.

    Parameters
    ----------
    model : pl.LightningModule
        Trained model.
    dataloader : DataLoader
        Validation or test DataLoader.
    threshold : float
        Threshold for multi-label predictions.
    multi_label : bool
        True if multi-label, False if multi-class.
    return_probs : bool
        If True, return probabilities.
    return_preds : bool
        If True, return predicted labels.

    Returns
    -------
    dict
        Metrics dictionary containing f1, accuracy, and optionally predictions/probabilities.
    """
    model.eval()
    all_preds, all_labels, all_probs = [], [], []

    with torch.no_grad():
        for batch in dataloader:
            # Unpack safely, ignoring extra items like x_lengths
            X_batch, y_batch, *_ = batch  

            logits = model(X_batch)

            if multi_label:
                probs = torch.sigmoid(logits)
                preds = (probs >= threshold).float()
            else:
                probs = torch.softmax(logits, dim=1)
                preds = torch.argmax(probs, dim=1)

            all_probs.append(probs.cpu().numpy())
            all_preds.append(preds.cpu().numpy())
            all_labels.append(y_batch.cpu().numpy())

    # Stack vertically
    all_probs = np.vstack(all_probs)
    all_preds = np.vstack(all_preds) if multi_label else np.concatenate(all_preds)
    all_labels = np.vstack(all_labels) if multi_label else np.argmax(np.vstack(all_labels), axis=1)

    metrics = {}

    if multi_label:
        # Multi-label metrics
        exact_acc = np.all(all_preds == all_labels, axis=1).mean()
        micro_acc = (all_preds == all_labels).mean()
        f1_macro = f1_score(all_labels, all_preds, average="macro", zero_division=0)
        metrics.update({
            "val_acc_exact": exact_acc,
            "val_acc_micro": micro_acc,
            "val_f1_macro": f1_macro
        })
        print("\n🧮 Multi-label Confusion Matrices (per class):")
        print(multilabel_confusion_matrix(all_labels, all_preds))
    else:
        # Multi-class metrics
        acc = (all_preds == all_labels).mean()
        f1_macro = f1_score(all_labels, all_preds, average="macro", zero_division=0)
        metrics.update({
            "val_acc": acc,
            "val_f1_macro": f1_macro
        })
        print("\n🧮 Classification Report:")
        print(classification_report(all_labels, all_preds, zero_division=0))

    if return_probs:
        metrics["probs"] = all_probs
    if return_preds:
        metrics["preds"] = all_preds

    return metrics


def train_model(
    data_csv,
    labels_csv=None,
    model_out_dir="models/saved_models",
    do_validation=True,
    return_mlb= True,
    seq_len=1,
    hidden_dim=20,  # <-- NEW: Renamed output_channel to hidden_dim
    lr=0.001,
    batch_size=32,
    max_epochs=200,
    save_model=False,
    return_val_accuracy=True,
    test_mode=False,
    tune_thresholds=False,
    include_no_label=False,
    label_weighting="none",
    scheduler_name="reduce_on_plateau",
    optimizer_params={"weight_decay": 0.01},
    scheduler_params={"factor": 0.2, "patience": 3},
    optimizer_name="adamw",
    dropout=0.2,
    multi_label = True
):
    """
    Train an LSTM classification model with labels coming from JSON (in-memory CSV).
    """

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_out = f"{model_out_dir}/lstm_model_class_{timestamp}.pt"
    meta_out = f"{model_out_dir}/lstm_meta_class_{timestamp}.pkl"


    pipeline = FeaturePipeline(
        steps=[
            make_step(add_candle_shape_features),
            # make_step(add_candle_rocp),
            # make_step(add_label_normalized_candles),
            make_step(drop_columns, cols_to_drop=["open","high","low","close","volume"]),
        ],
        # norm_methods={
            # "main": {
            #     "upper_shadow": "robust", "body": "standard", "lower_shadow": "standard",
            #     "upper_body_ratio": "standard", "lower_body_ratio": "standard",
            #     "upper_lower_body_ratio": "standard", "Candle_Color": "standard",
                
            # }
        #         "candle_shape": {
        #             "upper_shadow": "standard",
        #             "lower_shadow": "standard",
        #             "body": "standard",
        #             "color": "standard",
        #         }
        # },
        # window_norms={
        # "main": {"open_prop": "standard", "high_prop": "standard","low_prop": "standard", "close_prop": "standard"},},

        per_window_flags=[
        False, 
        False, 
        # True
                ],
    )
    
        # --- Get dataset(s) ---
    if do_validation:
        if return_mlb:
            train_ds, val_ds, df_labels, feature_columns, max_y, mlb = preprocess_sequences_csv_multilabels(
                data_csv, labels_csv,
                val_split=True,
                debug_sample=[0],
                feature_pipeline=pipeline,
        )
        else:    
            train_ds, val_ds, df_labels, feature_columns, max_y = preprocess_sequences_csv_multilabels(
                data_csv, labels_csv,
                val_split=True,
                debug_sample=[0],
                feature_pipeline=pipeline,
            )
    else:
        dataset, df_labels, feature_columns, max_y,mlb = preprocess_sequences_csv_multilabels(
            data_csv, labels_csv,
            val_split=False,
            debug_sample=[0],
        )

    # --- Model config ---
    num_classes = len(mlb.classes_)

    model = candle_cnn_multi_lstm_multi_class(
        hidden_dim=hidden_dim,  
        input_dim= train_ds.X_dict["main"][0].shape[1],
        num_classes=num_classes,
        lr=lr,
        scheduler_name=scheduler_name,
        optimizer_name=optimizer_name,
        optimizer_params=optimizer_params,
        scheduler_params=scheduler_params,
        dropout=dropout
    )

    init_args = {
        "hidden_dim": hidden_dim, 
        "num_classes": num_classes,
        "lr": lr,
        "scheduler_name": scheduler_name,
        "optimizer_params": optimizer_params,
        "scheduler_params": scheduler_params,
        "optimizer_name": optimizer_name,
        "dropout": dropout
    }

    model_class_info = {
        "module": model.__class__.__module__ ,
        "class": model.__class__.__name__ ,
        "init_args": init_args
    }
    # --- DataLoaders ---
    if do_validation:
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
        val_loader = DataLoader(val_ds, batch_size=batch_size,collate_fn=collate_batch)
    else:
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
        val_loader = None

    # --- Debug batch ---
    if test_mode:
        global df_seq
        df_seq = print_batch(train_loader, feature_columns, batch_idx=2)

    # --- Trainer ---
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="auto",
        devices=1,
        log_every_n_steps=10,
        fast_dev_run=test_mode,
    )

    trainer.fit(model, train_loader, val_loader)

    # --- Save model & metadata ---
    if save_model:
        os.makedirs(model_out_dir, exist_ok=True)
        trainer.save_checkpoint(model_out)
        joblib.dump({
            "hidden_dim": hidden_dim,
            # "output_channel": output_channel,
            "num_classes": num_classes,
            "seq_len": seq_len,
            "lr": lr,
            "feature_columns": feature_columns,
            "label_classes": mlb.classes_,
            "scalers": pipeline.scalers,
            "window_scalers": pipeline.window_scalers,
            "pipeline_config": pipeline.export_config(),
            "model_class_info": model_class_info,
            "include_no_label":include_no_label
        }, meta_out)
        print(f"\n✅ Model saved to {model_out}")
        print(f"✅ Meta saved to {meta_out}")

    if do_validation and return_val_accuracy:
        # Extract true labels
        y_true_val = np.vstack([y for *_, y in val_loader.dataset])
        y_true_val = np.array(y_true_val)

        # Run evaluation
        metrics = evaluate_model(
            model=model,
            dataloader=val_loader,
            multi_label=True,
            threshold=0.5,
            return_probs=True,
            return_preds=True
        )

        y_probs = np.array(metrics.get("probs"))
        y_pred_default = np.array(metrics.get("preds"))

        num_classes = y_pred_default.shape[1]

        # Convert y_true_val to one-hot if needed
        if y_true_val.shape[1] == 1 and multi_label:
            y_indices = y_true_val[:, 0].astype(int)

            # Drop invalid samples instead of clipping
            valid_mask = (y_indices >= 0) & (y_indices < num_classes)
            if not valid_mask.all():
                print(f"⚠️ Warning: Dropping {np.sum(~valid_mask)} samples with invalid labels")

            y_indices = y_indices[valid_mask]
            y_probs = y_probs[valid_mask]
            y_pred_default = y_pred_default[valid_mask]

            y_true_val_onehot = np.zeros((y_indices.shape[0], num_classes), dtype=int)
            y_true_val_onehot[np.arange(y_indices.shape[0]), y_indices] = 1
            y_true_val = y_true_val_onehot

        # Apply tuned thresholds if enabled
        if tune_thresholds:
            optimal_thresholds = tune_thresholds_nn(y_true=y_true_val, y_probs=y_probs)
            y_pred_tuned = (y_probs >= np.array(optimal_thresholds)[None, :]).astype(int)
        else:
            y_pred_tuned = y_pred_default

        # Ensure shapes match
        assert y_true_val.shape == y_pred_tuned.shape, f"Shapes differ: {y_true_val.shape} vs {y_pred_tuned.shape}"

        # Compute final metrics
        final_val_acc_exact = np.all(y_pred_tuned == y_true_val, axis=1).mean()
        final_val_acc_micro = (y_pred_tuned == y_true_val).mean()
        final_val_f1_macro = f1_score(y_true_val, y_pred_tuned, average="macro", zero_division=0)

        return {
            "val_acc_exact": float(final_val_acc_exact),
            "val_acc_micro": float(final_val_acc_micro),
            "val_f1_macro": float(final_val_f1_macro)
        }

if __name__ == "__main__":
    train_model(
        data_csv="/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv",
        labels_csv="/home/iatell/projects/meta-learning/data/dummy_multi_string_seq_dif.csv",  
        do_validation=True,
        save_model=False,

        test_mode= False,
        # output_channel = 20
    )


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | branches      | ModuleList       | 1.4 K  | train
1 | fusion_conv2d | Sequential       | 6.2 K  | train
2 | lstm          | LSTM             | 6.9 K  | train
3 | fc            | Linear           | 63     | train
4 | criterion     | CrossEntropyLoss | 0      | train
-----------------------------------------------------------
14.6 K    Trainable params
0         Non-trainable params
14.6 K    Total params
0.058     Total estimated model params size (MB)
23        Modules in train mode
0         Modules in eval mode


--- DEBUG SAMPLE 0 ---
Target (raw vector): [1. 0. 1.]
Active labels: ['a', 's']
[main] Shape: (4, 4)
[main] First few rows:
 [[0.00637838 0.14028078 0.08115927 0.3       ]
 [0.04009115 0.02701042 0.03372177 0.3       ]
 [0.00325818 0.0881668  0.03394962 0.7       ]
 [0.00191968 0.13889347 0.11387014 0.3       ]]
------------------------------


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.



🧮 Multi-label Confusion Matrices (per class):
[[[1 0]
  [3 0]]

 [[3 0]
  [1 0]]

 [[0 1]
  [0 3]]]


## xgboost

In [42]:
import os
import io
import joblib
import numpy as np
import xgboost as xgb
from datetime import datetime
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score, classification_report, multilabel_confusion_matrix
from preprocess.classification_multilabel_dif_seq import preprocess_sequences_csv_multilabels
from utils.multilabel_threshold_tuning import tune_thresholds

# -----------------------------
# ⚡ Evaluate Multi-Label XGBoost
# -----------------------------
def evaluate_multilabel_xgb(model, X_val, y_val, mlb, thresholds=None, return_probs=False):
    """
    Evaluate a multi-label XGBoost model with optional per-label thresholds.
    """
    # Predict probabilities per label
    y_probs = np.column_stack([est.predict_proba(X_val)[:, 1] for est in model.estimators_])

    # Apply thresholds
    if thresholds is None:
        thresholds = [0.5] * y_val.shape[1]
    y_pred = np.zeros_like(y_val)
    for i, t in enumerate(thresholds):
        y_pred[:, i] = (y_probs[:, i] >= t).astype(int)

    print("\n📊 Validation Report (Multi-label):")
    print(classification_report(y_val, y_pred, target_names=mlb.classes_, zero_division=0))

    print("\n🧮 Multi-label Confusion Matrices (per class):")
    mcm = multilabel_confusion_matrix(y_val, y_pred)
    for i, cls in enumerate(mlb.classes_):
        print(f"\nClass '{cls}':")
        print(mcm[i])

    # Metrics
    exact_match = np.all(y_pred == y_val, axis=1).mean()
    micro_acc = (y_pred == y_val).mean()
    f1_macro = f1_score(y_val, y_pred, average="macro", zero_division=0)

    print("\nExact match ratio:", exact_match)
    print("Micro accuracy (per-label):", micro_acc)
    print("Macro F1:", f1_macro)

    results = {
        "exact_match": exact_match,
        "micro_accuracy": micro_acc,
        "f1_macro": f1_macro,
    }
    if return_probs:
        results["y_probs"] = y_probs
        results["y_pred"] = y_pred
    return results


# -----------------------------
# ⚡ Training XGBoost for Multi-Label
# -----------------------------
def train_model_xgb_multilabel(
    data_csv,
    labels_json,
    model_out_dir="models/saved_models",
    do_validation=True,
    seq_len=1,
    n_estimators=200,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    save_model=False,
    return_val_accuracy=True,
    threshold_tuning=False,
):
    """
    Train a multi-label XGBoost model (one-vs-rest with MultiOutputClassifier).
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_out = f"{model_out_dir}/xgb_model_multilabel_{timestamp}.pkl"
    meta_out = f"{model_out_dir}/xgb_meta_multilabel_{timestamp}.pkl"

    # Convert JSON labels to CSV in-memory
    csv_string = json_to_csv_in_memory(labels_json)
    labels_csv = io.StringIO(csv_string)

    # --- Feature pipeline ---
    pipeline = FeaturePipeline(
        steps=[
            make_step(add_candle_shape_features),
            make_step(drop_columns, cols_to_drop=["open", "high", "low", "close", "volume"]),
        ],
        transformations={"mode": "flatten"},
        per_window_flags=[False, False],
    )

    # --- Preprocess ---
    if do_validation:
        X_train, y_train, X_val, y_val, df, feature_columns, mlb, label_weights = preprocess_sequences_csv_multilabels(
            data_csv,
            labels_csv,
            n_candles=seq_len,
            val_split=True,
            for_xgboost=True,
            feature_pipeline=pipeline,
        )
    else:
        X_train, y_train, df, feature_columns, mlb, label_weights = preprocess_sequences_csv_multilabels(
            data_csv,
            labels_csv,
            n_candles=seq_len,
            val_split=False,
            for_xgboost=True,
            feature_pipeline=pipeline,
        )
        X_val, y_val = None, None

    # --- Build XGBoost base learners ---
    xgb_models = []
    for w in label_weights:
        xgb_model = xgb.XGBClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            eval_metric="logloss",
            scale_pos_weight=w,
            use_label_encoder=False,
        )
        xgb_models.append(xgb_model)

    # Wrap into MultiOutputClassifier
    model = MultiOutputClassifier(xgb_models[0], n_jobs=-1)
    model.estimators_ = xgb_models
    model.fit(X_train, y_train)

    optimal_thresholds = None
    val_results = {}

    # --- Validation phase ---
    if do_validation and return_val_accuracy:
        # Default threshold evaluation
        val_results = evaluate_multilabel_xgb(model, X_val, y_val, mlb, thresholds=[0.5] * y_val.shape[1], return_probs=True)

        # Threshold tuning
        if threshold_tuning:
            optimal_thresholds = tune_thresholds(y_val, val_results["y_probs"])
            print("\n📌 Optimal thresholds per label:", dict(zip(mlb.classes_, optimal_thresholds)))

            val_results_tuned = evaluate_multilabel_xgb(model, X_val, y_val, mlb, thresholds=optimal_thresholds)
            val_results["optimal_thresholds"] = optimal_thresholds
            val_results["tuned"] = val_results_tuned

    # --- Save model ---
    if save_model:
        os.makedirs(model_out_dir, exist_ok=True)
        joblib.dump(model, model_out)
        joblib.dump(
            {
                "seq_len": seq_len,
                "label_classes": mlb.classes_,
                "optimal_thresholds": optimal_thresholds,
                "feature_columns": feature_columns,
                "scalers": pipeline.scalers,
                "window_scalers": pipeline.window_scalers,
                "pipeline_config": pipeline.export_config(),
            },
            meta_out,
        )
        print(f"✅ Model saved to {model_out}")
        print(f"✅ Meta saved to {meta_out}")

    return val_results if return_val_accuracy else model

if __name__ == "__main__":
    train_model(
        data_csv="/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv",
        labels_csv="/home/iatell/projects/meta-learning/data/dummy_multi_string_seq_dif.csv",  
        do_validation=True,
        save_model=False,

        test_mode= False,
        # output_channel = 20
    )


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | branches      | ModuleList       | 1.4 K  | train
1 | fusion_conv2d | Sequential       | 6.2 K  | train
2 | lstm          | LSTM             | 6.9 K  | train
3 | fc            | Linear           | 63     | train
4 | criterion     | CrossEntropyLoss | 0      | train
-----------------------------------------------------------
14.6 K    Trainable params
0         Non-trainable params
14.6 K    Total params
0.058     Total estimated model params size (MB)
23        Modules in train mode
0         Modules in eval mode


--- DEBUG SAMPLE 0 ---
Target (raw vector): [1. 0. 1.]
Active labels: ['a', 's']
[main] Shape: (4, 4)
[main] First few rows:
 [[0.00637838 0.14028078 0.08115927 0.3       ]
 [0.04009115 0.02701042 0.03372177 0.3       ]
 [0.00325818 0.0881668  0.03394962 0.7       ]
 [0.00191968 0.13889347 0.11387014 0.3       ]]
------------------------------


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.



🧮 Multi-label Confusion Matrices (per class):
[[[1 0]
  [2 1]]

 [[3 0]
  [1 0]]

 [[1 0]
  [2 1]]]


# Server

## cnn lstm

In [ ]:
import sys
import glob
import joblib
import torch
import os
import numpy as np
import pandas as pd
from pathlib import Path
from flask import Flask, request, jsonify, render_template

# ---------------- Imports ----------------
from servers.pre_process.multi_reg_dif_seq2 import (
    ServerPreprocess, build_pipeline_from_config
)
from models.LSTM.lstm_multi_label import LSTMMultiLabelClassifier  # adjust path/class name


# ---------------- Flask app ----------------
app = Flask(__name__)

# ---------------- Load model + meta ----------------
meta_files = glob.glob("/home/iatell/projects/meta-learning/play_grounds/models/saved_models/lstm_meta_class_*.pkl")
state_files = glob.glob("/home/iatell/projects/meta-learning/play_grounds/models/saved_models/lstm_model_class_*.pt")

if not meta_files or not state_files:
    raise FileNotFoundError("No multilabel model/meta found!")

meta_path = max(meta_files, key=os.path.getmtime)
state_path = max(state_files, key=os.path.getmtime)

print("Using meta:", meta_path)
print("Using state:", state_path)

meta = joblib.load(meta_path)
FEATURES = meta["feature_columns"]
LABEL_CLASSES = meta["label_classes"]
print("Features:", FEATURES)
print("Label classes:", LABEL_CLASSES)

# Init model
model_cls_info = meta["model_class_info"]
init_args = model_cls_info["init_args"]
model = LSTMMultiLabelClassifier.load_from_checkpoint(state_path, **init_args)
model.eval()

# ---------------- Pipeline ----------------
pipeline = build_pipeline_from_config(meta["pipeline_config"])
pipeline.scalers = meta["scalers"]
pipeline.window_scalers = meta["window_scalers"]

preproc = ServerPreprocess(feature_pipeline=pipeline)

# ---------------- Load dataset ----------------
DATA_FILE = "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv"  # change path as needed
df = pd.read_csv(DATA_FILE, parse_dates=["timestamp"])

# ---------------- Routes ----------------
@app.route("/")
def home():
    return render_template("multi_label.html")

@app.route("/get_and_add_data")
def get_and_add_data():
    # Convert timestamp column to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')  # or unit='ms' if needed
    # print("[DEBUG] df['timestamp'] head:", df['timestamp'].head())

    # Reindex to daily frequency and forward-fill missing
    dense = df.set_index('timestamp').asfreq('D').ffill()
    # print("[DEBUG] dense head:", dense.head())

    initial_seq_len = 21
    next_idx = request.args.get("idx", type=int)
    # print(f"[DEBUG] next_idx from request: {next_idx}")

    if next_idx is None:
        # --- First load ---
        if len(preproc.dataset) == 0:
            # print("[DEBUG] Adding initial candles to preproc.dataset")
            for _, row in dense.iloc[:initial_seq_len].iterrows():
                preproc.add_candle(row)

        candles = []
        for _, row in dense.iloc[:initial_seq_len].iterrows():
            # Use row.name which is the timestamp index
            time_val = int(row.name.timestamp())
            # print(f"[DEBUG] Candle time: {time_val}, high: {row.high}")
            candles.append({
                "time": time_val,
                "open": float(row.open),
                "high": float(row.high),
                "low": float(row.low),
                "close": float(row.close),
            })

        return jsonify({
            "initial_seq_len": initial_seq_len,
            "next_idx": initial_seq_len,
            "candles": candles
        })

    else:
        # --- Step forward ---
        if next_idx >= len(dense):
            # print("[DEBUG] End of data reached")
            return jsonify({"error": "End of data"}), 404

        row = dense.iloc[next_idx]
        time_val = int(row.name.timestamp())
        # print(f"[DEBUG] Step candle time: {time_val}, high: {row.high}")

        candle = {
            "time": time_val,
            "open": float(row.open),
            "high": float(row.high),
            "low": float(row.low),
            "close": float(row.close),
        }

        preproc.add_candle(row)
        # print(f"[DEBUG] Added candle to preproc, next_idx will be {next_idx + 1}")

        return jsonify({
            "next_idx": next_idx + 1,
            "candle": candle
        })


@app.route("/predict", methods=["POST"])
def predict():
    # Use seq_len from meta (training config)
    seq_len = meta.get("seq_len")
    if not seq_len or not isinstance(seq_len, int):
        return jsonify({"error": "Server meta is missing a valid 'seq_len'"}), 500

    # Build sequence from file / preprocessor
    try:
        seq_dict = preproc.prepare_seq(seq_len)  
    except ValueError as e:
        return jsonify({"error": str(e)}), 400

    # Convert dict of DataFrames → tensors
    dict_x = {k: torch.from_numpy(v.values.astype(np.float32)).unsqueeze(0)
              for k, v in seq_dict.items()}
    lengths = torch.tensor([seq_len], dtype=torch.long)
    with torch.no_grad():
        y_probs = model(dict_x["main"]).sigmoid().cpu().numpy()[0]

    # Decode labels above threshold
    threshold = 0.5
    predicted_labels = [LABEL_CLASSES[i] for i, p in enumerate(y_probs) if p >= threshold]

    # Get last candle time directly (it's already an int timestamp)
    return jsonify({
        "raw_probs": dict(zip(LABEL_CLASSES, y_probs.tolist())),
        "predicted_labels": predicted_labels
    })


# ---------------- Run ----------------
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


## xgboost

In [ ]:
import sys
import glob
import joblib
import torch
import os
import numpy as np
import pandas as pd
from pathlib import Path
from flask import Flask, request, jsonify, render_template

# ---------------- Imports ----------------
from servers.pre_process.multi_reg_dif_seq2 import (
    ServerPreprocess, build_pipeline_from_config
)
from models.LSTM.lstm_multi_label import LSTMMultiLabelClassifier  # adjust path/class name


# ---------------- Flask app ----------------
app = Flask(__name__)

# ---------------- Load model + meta ----------------
meta_files = glob.glob("/home/iatell/projects/meta-learning/play_grounds/models/saved_models/lstm_meta_class_*.pkl")
state_files = glob.glob("/home/iatell/projects/meta-learning/play_grounds/models/saved_models/lstm_model_class_*.pt")

if not meta_files or not state_files:
    raise FileNotFoundError("No multilabel model/meta found!")

meta_path = max(meta_files, key=os.path.getmtime)
state_path = max(state_files, key=os.path.getmtime)

print("Using meta:", meta_path)
print("Using state:", state_path)

meta = joblib.load(meta_path)
FEATURES = meta["feature_columns"]
LABEL_CLASSES = meta["label_classes"]
print("Features:", FEATURES)
print("Label classes:", LABEL_CLASSES)

# Init model
model_cls_info = meta["model_class_info"]
init_args = model_cls_info["init_args"]
model = LSTMMultiLabelClassifier.load_from_checkpoint(state_path, **init_args)
model.eval()

# ---------------- Pipeline ----------------
pipeline = build_pipeline_from_config(meta["pipeline_config"])
pipeline.scalers = meta["scalers"]
pipeline.window_scalers = meta["window_scalers"]

preproc = ServerPreprocess(feature_pipeline=pipeline)

# ---------------- Load dataset ----------------
DATA_FILE = "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv"  # change path as needed
df = pd.read_csv(DATA_FILE, parse_dates=["timestamp"])

# ---------------- Routes ----------------
@app.route("/")
def home():
    return render_template("multi_label.html")

@app.route("/get_and_add_data")
def get_and_add_data():
    # Convert timestamp column to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')  # or unit='ms' if needed
    # print("[DEBUG] df['timestamp'] head:", df['timestamp'].head())

    # Reindex to daily frequency and forward-fill missing
    dense = df.set_index('timestamp').asfreq('D').ffill()
    # print("[DEBUG] dense head:", dense.head())

    initial_seq_len = 21
    next_idx = request.args.get("idx", type=int)
    # print(f"[DEBUG] next_idx from request: {next_idx}")

    if next_idx is None:
        # --- First load ---
        if len(preproc.dataset) == 0:
            # print("[DEBUG] Adding initial candles to preproc.dataset")
            for _, row in dense.iloc[:initial_seq_len].iterrows():
                preproc.add_candle(row)

        candles = []
        for _, row in dense.iloc[:initial_seq_len].iterrows():
            # Use row.name which is the timestamp index
            time_val = int(row.name.timestamp())
            # print(f"[DEBUG] Candle time: {time_val}, high: {row.high}")
            candles.append({
                "time": time_val,
                "open": float(row.open),
                "high": float(row.high),
                "low": float(row.low),
                "close": float(row.close),
            })

        return jsonify({
            "initial_seq_len": initial_seq_len,
            "next_idx": initial_seq_len,
            "candles": candles
        })

    else:
        # --- Step forward ---
        if next_idx >= len(dense):
            # print("[DEBUG] End of data reached")
            return jsonify({"error": "End of data"}), 404

        row = dense.iloc[next_idx]
        time_val = int(row.name.timestamp())
        # print(f"[DEBUG] Step candle time: {time_val}, high: {row.high}")

        candle = {
            "time": time_val,
            "open": float(row.open),
            "high": float(row.high),
            "low": float(row.low),
            "close": float(row.close),
        }

        preproc.add_candle(row)
        # print(f"[DEBUG] Added candle to preproc, next_idx will be {next_idx + 1}")

        return jsonify({
            "next_idx": next_idx + 1,
            "candle": candle
        })


@app.route("/predict", methods=["POST"])
def predict():
    # Use seq_len from meta (training config)
    seq_len = meta.get("seq_len")
    if not seq_len or not isinstance(seq_len, int):
        return jsonify({"error": "Server meta is missing a valid 'seq_len'"}), 500

    # Build sequence from file / preprocessor
    try:
        seq_dict = preproc.prepare_seq(seq_len)  
    except ValueError as e:
        return jsonify({"error": str(e)}), 400

    # Convert dict of DataFrames → tensors
    dict_x = {k: torch.from_numpy(v.values.astype(np.float32)).unsqueeze(0)
              for k, v in seq_dict.items()}
    lengths = torch.tensor([seq_len], dtype=torch.long)
    with torch.no_grad():
        y_probs = model(dict_x["main"]).sigmoid().cpu().numpy()[0]

    # Decode labels above threshold
    threshold = 0.5
    predicted_labels = [LABEL_CLASSES[i] for i, p in enumerate(y_probs) if p >= threshold]

    # Get last candle time directly (it's already an int timestamp)
    return jsonify({
        "raw_probs": dict(zip(LABEL_CLASSES, y_probs.tolist())),
        "predicted_labels": predicted_labels
    })


# ---------------- Run ----------------
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


# Tuning

## cnn lstm

In [ ]:
import os
import torch
from datetime import datetime

from ray import tune, air
from ray.tune.schedulers import ASHAScheduler

from utils.flatten_config import flatten_config
from utils.resoure_usage import resource_usage

# --- Data paths ---
data_csv = "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv"
labels_json = "/home/iatell/projects/meta-learning/data/candle_labels.json"


def train_model_tune(config):
    """
    Single Ray Tune trial.
    """
    resource_usage()  # Show hardware usage
    
    # --- Fixed arguments ---
    fixed_args = dict(
        data_csv=data_csv,
        labels_json=labels_json,
        do_validation=True,
        model_out_dir="models/tuned",
        return_val_accuracy=True,
        save_model=False,
    )
    
    # --- Build training args ---
    train_args = fixed_args.copy()

    # Model hyperparams
    train_args["seq_len"] = config["seq_len"]
    train_args["hidden_dim"] = config["hidden_dim"]

    # Training hyperparams
    train_args["lr"] = config["lr"]
    train_args["batch_size"] = config["batch_size"]
    train_args["max_epochs"] = config["max_epochs"]
    train_args["dropout"] = config["dropout"]

    # Optimizer
    train_args["optimizer_name"] = config["optimizer_name"]
    if train_args["optimizer_name"] in ["adamw", "adam"]:
        train_args["optimizer_params"] = {"weight_decay": config["weight_decay"]}
    else:
        train_args["optimizer_params"] = {}

    # Scheduler
    train_args["scheduler_name"] = config["scheduler_name"]
    if train_args["scheduler_name"] == "reduce_on_plateau":
        train_args["scheduler_params"] = {
            "factor": config["scheduler_params"]["factor"],
            "patience": config["scheduler_params"]["patience"]
        }
    elif train_args["scheduler_name"] == "cosine":
        train_args["scheduler_params"] = {
            "T_max": config["scheduler_params"]["T_max"],
            "eta_min": config["scheduler_params"]["eta_min"]
        }
    else:
        train_args["scheduler_params"] = {}

    # --- Run training ---
    metrics = train_model(**train_args)

    # Report back to Ray Tune
    tune.report(metrics)


def run_tuning(save_model=False):
    """Hyperparameter tuning for CNN-LSTM with Ray Tune."""
    
    search_space = {
        # Model
        "seq_len": tune.choice([1, 5, 10, 20]),
        "hidden_dim": tune.choice([32, 64, 128, 256]),
        "dropout": tune.uniform(0.1, 0.5),

        # Training
        "lr": tune.loguniform(1e-4, 1e-3),
        "batch_size": tune.choice([32, 64, 128]),
        "max_epochs": tune.choice([50, 100, 150]),

        # Optimizer
        "optimizer_name": tune.choice(["adamw", "adam"]),
        "weight_decay": tune.loguniform(1e-5, 1e-2),

        # Scheduler
        "scheduler_name": tune.choice(["reduce_on_plateau", "cosine", "none"]),
        "scheduler_params": {
            "factor": tune.uniform(0.1, 0.5),
            "patience": tune.choice([3, 5, 10]),
            "T_max": tune.choice([10, 20, 50]),
            "eta_min": tune.loguniform(1e-6, 1e-4),
        },
    }

    scheduler = ASHAScheduler(
        metric="f1_macro",
        mode="max",
        grace_period=5,
        reduction_factor=2,
    )

    # --- Add timestamp to run name ---
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_name = f"cnn_lstm_tuning_{timestamp}"

    tuner = tune.Tuner(
        tune.with_resources(train_model_tune, {"cpu": 1, "gpu": 1}),
        param_space=search_space,
        tune_config=tune.TuneConfig(
            scheduler=scheduler,
            num_samples=10,
        ),
        run_config=air.RunConfig(
            name=run_name,
            storage_path="/home/iatell/projects/meta-learning/tune_logs",
        ),
    )

    results = tuner.fit()

    # --- Best trial ---
    best_result = results.get_best_result(metric="f1_macro", mode="max")
    print("\n🏆 Best Config:", best_result.config)
    print(f"Best F1 Macro: {best_result.metrics['f1_macro']:.4f}")

    # --- Retrain with best config ---
    if save_model:
        print("\n🔁 Retraining best model on full dataset for saving...")
        retrain_args = {
            "data_csv": data_csv,
            "labels_json": labels_json,
            "do_validation": False,
            "model_out_dir": "models/saved_models",
            "save_model": True,
            "seq_len": best_result.config.get("seq_len", 1),
            "hidden_dim": best_result.config["hidden_dim"],
            "lr": best_result.config["lr"],
            "batch_size": best_result.config["batch_size"],
            "max_epochs": best_result.config["max_epochs"],
            "dropout": best_result.config["dropout"],
            "optimizer_name": best_result.config["optimizer_name"],
            "scheduler_name": best_result.config["scheduler_name"],
            "optimizer_params": {"weight_decay": best_result.config["weight_decay"]},
            "scheduler_params": best_result.config["scheduler_params"],
        }
        train_model(**retrain_args)


if __name__ == "__main__":
    run_tuning(save_model=True)


## xgboost

In [ ]:
import os
import torch
from utils.flatten_config import flatten_config
from utils.resoure_usage import resource_usage
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray import air
from datetime import datetime

def train_model_tune(config):
    """
    Single Ray Tune trial for XGB Multilabel model.
    """
    resource_usage()  # Show current hardware usage

    # Always required data sources
    data_csv = "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv"
    labels_json = "/home/iatell/projects/meta-learning/data/candle_labels.json"

    # Fixed args
    fixed_args = dict(
        data_csv=data_csv,
        labels_json=labels_json,
        do_validation=True,
        model_out_dir="models/tuned",
        return_val_accuracy=True,
        save_model=False,
    )

    # Flatten (in case config has nested dicts)
    flat_config = flatten_config(config)

    # Merge everything
    train_args = {**fixed_args, **flat_config}

    # Train and collect metrics
    metrics = train_model_xgb_multilabel(**train_args)

    # Report to Ray Tune
    tune.report(metrics)


def run_tuning(save_model=False):
    """Hyperparameter tuning for XGB Multilabel with Ray Tune."""

    search_space = {
        # Sequence length
        # "seq_len": tune.choice([1, 3, 5, 10, 20]),

        # Core XGB hyperparameters
        "n_estimators": tune.choice([100, 200, 400, 800]),
        # "max_depth": tune.choice([3, 6, 9]),
        # "learning_rate": tune.loguniform(1e-3, 0.3),   # typical XGB LR range
        "subsample": tune.uniform(0.6, 1.0),
        "colsample_bytree": tune.uniform(0.6, 1.0),

        # Label handling
        # "label_weighting": tune.choice(["none", "scale_pos"]),
        # "include_no_label": tune.choice([False, True]),

        # Threshold tuning
        # "threshold_tuning": tune.choice([False, True]),
    }

    scheduler = ASHAScheduler(
        metric="exact_match",   # use micro accuracy from your return dict
        mode="max",
        grace_period=1,
        reduction_factor=2
    )
    # --- Add timestamp to run name ---
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_name = f"xgb_multilabel_tuning_{timestamp}"

    tuner = tune.Tuner(
        train_model_tune,
        param_space=search_space,
        tune_config=tune.TuneConfig(
            scheduler=scheduler,
            num_samples=20,   # try more samples, adjust depending on compute
        ),
        run_config=air.RunConfig(
            name=run_name,
            storage_path="/home/iatell/projects/meta-learning/tune_logs",
        ),
    )

    results = tuner.fit()

    # Best trial
    best_result = results.get_best_result(metric="exact_match", mode="max")
    print("\n🏆 Best Config:", best_result.config)
    print(f"Best Micro Accuracy: {best_result.metrics['exact_match']:.4f}")
    # Optional: retrain best model on full data and save
    if save_model:
        print("\n🔁 Retraining best XGB model on full dataset for saving...")

        # Re-train using the best configuration found
        final_model_metrics = train_model_xgb_multilabel(
            data_csv="/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv",
            labels_json="/home/iatell/projects/meta-learning/data/candle_labels.json",
            do_validation=True,
            return_val_accuracy=True,
            model_out_dir="models/saved_models",
            save_model=True,  # <--- actually save model here
            seq_len=best_result.config.get("seq_len", 5),
            n_estimators=best_result.config.get("n_estimators", 200),
            max_depth=best_result.config.get("max_depth", 6),
            learning_rate=best_result.config.get("learning_rate", 0.05),
            subsample=best_result.config.get("subsample", 0.8),
            colsample_bytree=best_result.config.get("colsample_bytree", 0.8),
            label_weighting=best_result.config.get("label_weighting", "none"),
            include_no_label=best_result.config.get("include_no_label", False),
            threshold_tuning=best_result.config.get("threshold_tuning", False),
        )

        print("\n💾 Final saved model metrics:", final_model_metrics)
if __name__ == "__main__":
    run_tuning(save_model=False)
